## project

In [1]:
%env DD_TOKEN=https://oapi.dingtalk.com/robot/send?access_token=1003a3d8a6e724e255d9b28b5d4a540cd8dbdd2d2fa2163ed09cc77f7c46841d

env: DD_TOKEN=https://oapi.dingtalk.com/robot/send?access_token=1003a3d8a6e724e255d9b28b5d4a540cd8dbdd2d2fa2163ed09cc77f7c46841d


In [2]:
from zxftools.llms import get_llm
import os
import subprocess

In [3]:
import re
def get_code(result):
    code_match = re.search(r'### 3. Function\n\n```python(.*?)```',result,flags = re.DOTALL)
    return code_match.group(1)

In [4]:
def write_code(main_path,llm):
    with open(os.path.join(main_path,'test.py'),'r') as f:
        unittest_code = f.read()
    result = llm.complete(f"""
You are a python programer helping user to deal problem by create a python function.
Following '===' is the unittest code. 
Only use the unittest code between first and second '===' to accomplish the task above, do not Mixed into the python function.

===
{unittest_code}
===

Never forget that your functions should be robust and adaptable. your goal is to pass the unit tests.

Your results should be kept in four parts: 
    1 your deconstruction and understanding of the problem
    2 your function design ideas (pseudocode)
    3 your formal function
    4 your function call examples
    
Example:

### 1. Understand
The task is xxx
### 2. Design
1. xxxx
2. xxxx
3. xxxx
### 3. Function

```python
import re

def prepend_to_file(file_path, data_to_prepend,new_file_path=None):
    
    if new_file_path is None:
        new_file_path = file_path
    # 读取原始文件内容
    with open(file_path, 'r', encoding='utf-8') as file:
        original_content = file.read()
    
    # 将新数据添加到原始内容前面
    updated_content = data_to_prepend +'\n'+ original_content
    
    # 将更新后的内容写回文件
    with open(new_file_path, 'w', encoding='utf-8') as file:
        file.write(updated_content)
```
### 4. Function Call Examples
```python
xxx
```
    
""").text
    return get_code(result)

In [5]:
def rewrite_code(code,error_info,llm):
    result = llm.complete(f"""
You are a python programer helping user to deal problem by check python functions ,eliminate bugs and fix errors.
Following '===' is the user's code and execution error result. 
Only use the user's code and execution error result between first and second '===' to accomplish the task above, do not Mixed into the python function.

===
{code}


{error_info}
===



Never forget that your functions should be robust and adaptable. your goal is to pass the unit tests.

Your results should be kept in four parts: 
    1 your deconstruction and understanding of the problem
    2 your function design ideas (pseudocode)
    3 your formal function
    4 your function call examples
    
Example:

### 1. Understand
The task is xxx
### 2. Design
1. xxxx
2. xxxx
3. xxxx
### 3. Function

```python
import re

def prepend_to_file(file_path, data_to_prepend,new_file_path=None):
    
    if new_file_path is None:
        new_file_path = file_path
    # 读取原始文件内容
    with open(file_path, 'r', encoding='utf-8') as file:
        original_content = file.read()
    
    # 将新数据添加到原始内容前面
    updated_content = data_to_prepend +'\n'+ original_content
    
    # 将更新后的内容写回文件
    with open(new_file_path, 'w', encoding='utf-8') as file:
        file.write(updated_content)
```
### 4. Function Call Examples
```python
xxx
```
    
""").text
    return get_code(result)

In [6]:
def prepend_to_file(file_path, data_to_prepend,new_file_path=None):
    
    if new_file_path is None:
        new_file_path = file_path
    # 读取原始文件内容
    with open(file_path, 'r', encoding='utf-8') as file:
        original_content = file.read()
    
    # 将新数据添加到原始内容前面
    updated_content = data_to_prepend +'\n'+ original_content
    
    # 将更新后的内容写回文件
    with open(new_file_path, 'w', encoding='utf-8') as file:
        file.write(updated_content)
        
    cc = """
if __name__ == '__main__':
    unittest.main()
"""
    with open(new_file_path,'a', encoding='utf-8') as file:
        file.write(cc)


In [7]:
from zxftools.project import DDMessage

In [8]:
robot = DDMessage()

In [9]:
llm = get_llm(model='gpt-4o',temperature=0.01)

## label_input_position_info

### init

In [10]:
# Unit tests

### RUN

In [11]:
func_name = 'wake_function'

In [12]:
main_path = os.path.join('./function/',func_name)

In [13]:
if not os.path.exists(main_path):
    os.makedirs(main_path)

In [14]:
%%writefile {main_path}/test.py
import unittest

class TestWakeFunction(unittest.TestCase):
    
    def test_wake_function_with_high_decibels(self):
        input_decibels = 80  # 假设80是阈值
        result = wake_function(input_decibels, "no_wake_word")
        self.assertEqual(result.sound, "bi")
        self.assertEqual(result.light, "on")
        self.assertEqual(result.status, "working")
        self.assertEqual(result.duration, 3600)  # 秒
    
    def test_wake_function_with_low_decibels(self):
        input_decibels = 50  # 阈值以下
        result = wake_function(input_decibels, "no_wake_word")
        self.assertEqual(result.sound, "none")
        self.assertEqual(result.light, "off")
        self.assertEqual(result.status, "idle")
        self.assertEqual(result.duration, 0)
    
    def test_wake_function_with_wake_word(self):
        input_decibels = 40  # 阈值以下
        result = wake_function(input_decibels, "wake_word")
        self.assertEqual(result.sound, "bi")
        self.assertEqual(result.light, "on")
        self.assertEqual(result.status, "working")
        self.assertEqual(result.duration, 3600)  # 秒
    
    def test_wake_function_with_no_wake_word(self):
        input_decibels = 40  # 阈值以下
        result = wake_function(input_decibels, "no_wake_word")
        self.assertEqual(result.sound, "none")
        self.assertEqual(result.light, "off")
        self.assertEqual(result.status, "idle")
        self.assertEqual(result.duration, 0)
    
    def test_wake_function_with_high_decibels_and_wake_word(self):
        input_decibels = 80  # 假设80是阈值
        result = wake_function(input_decibels, "wake_word")
        self.assertEqual(result.sound, "bi")
        self.assertEqual(result.light, "on")
        self.assertEqual(result.status, "working")
        self.assertEqual(result.duration, 3600)  # 秒
    
    def test_wake_function_on_threshold(self):
        input_decibels = 80  # 正好等于阈值
        result = wake_function(input_decibels, "no_wake_word")
        self.assertEqual(result.sound, "bi")
        self.assertEqual(result.light, "on")
        self.assertEqual(result.status, "working")
        self.assertEqual(result.duration, 3600)  # 秒

if __name__ == '__main__':
    unittest.main()


Writing ./function/wake_function/test.py


In [15]:
verbose = True

In [16]:
for i in range(3):
    if i == 0:
        python_code = write_code(main_path,llm)
        prepend_to_file(os.path.join(main_path,'test.py'), 
                        python_code,
                        os.path.join(main_path,f'test_run_{i}.py'))
        unittest_result = subprocess.run(f"python {main_path}/test_run_{i}.py", shell=True, text=True, capture_output=True)
    else:
        with open(f'{main_path}/test_run_{i-1}.py','r') as f:
            code = f.read()
        print(f'第{i}轮修改')
        print(unittest_result.stderr)

        python_code = rewrite_code(code,unittest_result.stderr,llm)
        prepend_to_file(os.path.join(main_path,'test.py'), 
                        python_code,
                        os.path.join(main_path,f'test_run_{i}.py'))
        unittest_result = subprocess.run(f"python {main_path}/test_run_{i}.py", shell=True, text=True, capture_output=True)

    if unittest_result.returncode == 0:
        func_name = main_path.split('/')[-1]
        func_name = "completed_function"
        with open(os.path.join(main_path,f"{func_name}.py"),'w') as f:
            f.write(python_code.replace('func',func_name))
        robot.send(f'函数{func_name} 的编写以完成,已保存至{os.path.join(main_path,f"{func_name}.py")}中')
        break  # 如果条件为假，跳出循环

In [ ]:
%load {main_path}/completed_function.py

In [19]:
# %load ./function/wake_function/completed_function.py

class WakeResult:
    def __init__(self, sound, light, status, duration):
        self.sound = sound
        self.light = light
        self.status = status
        self.duration = duration

def wake_completed_functiontion(decibels, wake_word):
    if decibels >= 80 or wake_word == "wake_word":
        return WakeResult(sound="bi", light="on", status="working", duration=3600)
    else:
        return WakeResult(sound="none", light="off", status="idle", duration=0)
